In [11]:
import pandas as pd, numpy as np,time, datetime, csv, os
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
list_of_agencies = sorted(list(os.walk('gtfs'))[0][1])

In [3]:
# define functions

def minSinceMidnight(t):
    if type(t) == str:
        tt = t.split(':')
        msm = (int(tt[0])*60) + int(tt[1]) + (int(tt[2])/60)
    else:
        msm = t
    return msm

## REWROTE on 9/20 TO DEAL WITH DUPLICATE TIMES

def interpolator(trip):
    
    lsst = list(stop_times[stop_times['trip_id'] == trip]['arrival_time'])
    lsst = [minSinceMidnight(x) for x in lsst]
    
    last_scheduled_time = lsst[0]
    lst_index = 0
    
    values = [last_scheduled_time]
    
    for x in range(1,len(lsst)):
        #print(x)
        if pd.isnull(lsst[x]) == False:
            #idx = lsst.index(x)
            #print(idx)
            yy = list(np.linspace(last_scheduled_time, lsst[x], num=(x-lst_index+1)))

            values += yy[1:]
            last_scheduled_time = lsst[x]
            lst_index = x
        else:
            pass
    return values # returns linear interpolation of times

In [14]:
# unchanging data

# days to evaluate
weekday = "20190201"
saturday = "20190202"
sunday = "20190203"

days_of_week = {0:"monday", 1:"tuesday", 2:"wednesday", 3:"thursday", 4:"friday", 5:"saturday", 6:"sunday"}

# modes
bus_types = [3]
rail_types = [0,1,2]
ferry_types = [4]

# queries to filter by various time criteria
# strings for pd.DF.query

am_peak_query = "interpolated >= 360 and interpolated <= 600"
pm_peak_query = "interpolated >= 900 and interpolated <= 1140"
weekday_range_query = "interpolated >= 360 and interpolated <= 1320"
saturday_range_query = "interpolated >= 480 and interpolated <= 1320"
sunday_range_query = "interpolated >= 480 and interpolated <= 1320"

# defines the schema for the dict object holding headways

dataHolderSpec = {
                  "name":'',
                  "longitude":np.NaN,
                  "latitude":np.NaN,
                0: {
                    "served_by": [],
                    "AM Peak": np.NaN,
                    "PM Peak": np.NaN,
                    "Weekdays": np.NaN,
                    "Saturday": np.NaN,
                    "Sunday": np.NaN
                     },
                 1: {
                    "served_by": [],
                    "AM Peak": np.NaN,
                    "PM Peak": np.NaN,
                    "Weekdays": np.NaN,
                    "Saturday": np.NaN,
                    "Sunday": np.NaN
                     }
                 }


# starting index
start_index = 0

In [13]:
path

'gtfs/long-beach-transit--704/'

In [12]:
%%time

dayfile = open("output/v3/info.txt","w") 
 
dayfile.write()
dayfile.close() 

errors = pd.DataFrame(columns=["agency","path", "error"])

for agency in list_of_agencies[:]:
    path = "gtfs/" + agency + "/"
    #print(path)
    
    startTime = time.time()
    
    
    # LOAD GTFS DATA
    agency_name = pd.read_csv(path + 'agency.txt')['agency_name'][0]
    agency_name = agency_name.replace("/","-")
    print(agency_name, list_of_agencies.index(agency)+1, "of", len(list_of_agencies))

    trips = pd.read_csv(path + 'trips.txt', dtype={"trip_id":str, "route_id":str})
    #print(len(trips), "trips")

    routes = pd.read_csv(path + 'routes.txt',dtype={"route_id":str})
    #print(len(routes), "routes")

    stops = pd.read_csv(path + 'stops.txt', dtype={"stop_id":str})
    #print(len(stops), "stops")

    stop_times = pd.read_csv(path + 'stop_times.txt', dtype={"stop_id":str})
    #print(len(stop_times), "stop times")

    if os.path.isfile(path + "calendar.txt"):
        calendar = pd.read_csv(path + "calendar.txt")
        #print(len(calendar), "schedules")
        calend = True
        
        if sum(calendar[["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]].sum()) == 0:
            # build a calendar
            calend = True
            
            calendar_dates= pd.read_csv(path + "calendar_dates.txt")
            calendar_dates["day_of_week"] = [datetime.datetime(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])).weekday() for x in calendar_dates["date"]]

            calendar = pd.crosstab(calendar_dates["service_id"], calendar_dates["day_of_week"]).rename(days_of_week, axis=1).reset_index()

            service_id_df = []

            for service_id in calendar_dates["service_id"].unique():

                tempServiceID_df = calendar_dates[calendar_dates["service_id"] == service_id]
                minDate = tempServiceID_df["date"].min()
                maxDate = tempServiceID_df["date"].max()
                service_id_df.append(pd.DataFrame([[service_id, minDate,maxDate]], columns=["service_id", "start_date","end_date"]))
    
    
            calendar = calendar.merge(pd.concat(service_id_df, ignore_index=True))
            for date in ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]:
                calendar[date] = [1 if x > 0 else 0 for x in calendar[date]]
    else:
        calend = False

    calendar_dates = pd.read_csv(path + "calendar_dates.txt")
    #print(len(calendar_dates), "exception dates")

    # READ AND DETERMINE CALENDARS

    # create dictionary object with exceptions dates for different types of service

    exceptions = {}
    
    for x in range(len(calendar_dates)):
        exception_date = str(calendar_dates['date'][x])
        exception_service_id = calendar_dates['service_id'][x]
        exception_type = calendar_dates['exception_type'][x]

        if exception_date not in exceptions:
            exceptions[exception_date] = {1:[],2:[]}
            exceptions[exception_date][exception_type].append(exception_service_id)
        else:
            exceptions[exception_date][exception_type].append(exception_service_id)

    # GENERATES service_ids IN USE ON SPECIFIED DAY OF WEEK
    # ALSO ENSURES THAT SCHEDULES ARE ACTIVE DURING SPECIFIED TIME FRAME

    weekday_day = days_of_week[datetime.datetime(int(weekday[:4]), int(weekday[4:6]), int(weekday[6:])).weekday()]
    saturday_day = days_of_week[datetime.datetime(int(saturday[:4]), int(saturday[4:6]), int(saturday[6:])).weekday()]
    sunday_day = days_of_week[datetime.datetime(int(sunday[:4]), int(sunday[4:6]), int(sunday[6:])).weekday()]

    # this if-else statement checks to make sure that calendar.txt file is up to date
    # if none of the service_id are currently active, we take them anyway
    # otherwise, we filter out any inactive service_id
    
    # only use start dates for service filtering
    weekday_query = "(%s == 1) & (%s >= start_date)" % (weekday_day, weekday)
    saturday_query = "(%s == 1) & (%s >= start_date)" % (saturday_day, saturday)
    sunday_query = "(%s == 1) & (%s >= start_date)" % (sunday_day, sunday)
    
    
    '''try:
        if len([x for x in list(calendar['end_date']) if x > np.min([int(weekday), int(saturday), int(sunday)])]) == 0:
            print("just start date")
            weekday_query = "(%s == 1) & (%s >= start_date)" % (weekday_day, weekday)
            saturday_query = "(%s == 1) & (%s >= start_date)" % (saturday_day, saturday)
            sunday_query = "(%s == 1) & (%s >= start_date)" % (sunday_day, sunday)
        else:
            print("end date too")
            weekday_query = "(%s == 1) & (%s >= start_date) & (%s <= end_date)" % (weekday_day, weekday, weekday)
            saturday_query = "(%s == 1) & (%s >= start_date) & (%s <= end_date)" % (saturday_day, saturday, saturday)
            sunday_query = "(%s == 1) & (%s >= start_date) & (%s <= end_date)" % (sunday_day, sunday, sunday)
    except NameError:
        # the Long Beach Exception
        weekday_query = "(%s == 1) & (%s >= start_date) & (%s <= end_date)" % (weekday_day, weekday, weekday)
        saturday_query = "(%s == 1) & (%s >= start_date) & (%s <= end_date)" % (saturday_day, saturday, saturday)
        sunday_query = "(%s == 1) & (%s >= start_date) & (%s <= end_date)" % (sunday_day, sunday, sunday)'''

    # LISTS OF SERVICE_IDs FOR USE IN ANALYSIS
    if calend == True:
        weekday_service = list(calendar.query(weekday_query)['service_id'])
        saturday_service = list(calendar.query(saturday_query)['service_id'])
        sunday_service = list(calendar.query(sunday_query)['service_id'])
    elif calend == False:
        weekday_service = exceptions[weekday][1]
        saturday_service = exceptions[saturday][1]
        sunday_service = exceptions[sunday][1]
    # create dictionary object to hold information about stops
    # dictionary will be used for quick access
    
    if len(weekday_service) == 0 :
        if sum(calendar[weekday_day]) == 0:
            #print("no agency weekday service")
            errors = errors.append(pd.DataFrame([[agency_name, path,"no agency weekday service"]], columns=["agency", "path","error"]), ignore_index=True)
        else:
            #print("missing weekday service")
            errors = errors.append(pd.DataFrame([[agency_name, path,"missing weekday service"]], columns=["agency", "path","error"]), ignore_index=True)
    if len(saturday_service) == 0:
        if sum(calendar["saturday"]) == 0:
            #print("no agency saturday service")
            errors = errors.append(pd.DataFrame([[agency_name, path, "no agency saturday service"]], columns=["agency", "path","error"]), ignore_index=True)
        else:
            #print("missing weekday service") 
            errors = errors.append(pd.DataFrame([[agency_name, path,"missing saturday service"]], columns=["agency", "path","error"]), ignore_index=True)
    if len(sunday_service) == 0:
        if sum(calendar["sunday"]) == 0:
            #print("no agency sunday service")
            errors = errors.append(pd.DataFrame([[agency_name, path,"no agency sunday service"]], columns=["agency", "path","error"]), ignore_index=True)
        else:
            #print("missing weekday service")
            errors = errors.append(pd.DataFrame([[agency_name, path,"missing sunday service"]], columns=["agency", "path","error"]), ignore_index=True)
    
    # create the fully-formed stops file
    if "direction_id" in trips.columns:
        trip_info = trips[["route_id", "service_id", "trip_id", "direction_id"]]
    else:
        trip_info = trips[["route_id", "service_id", "trip_id"]]
    route_info = routes[['route_id', 'route_type']]

    trip_route_merged = trip_info.merge(route_info, on="route_id")

    # reorder columns
    if "direction_id" in trips.columns:
        trip_route_merged = trip_route_merged[["trip_id", "route_id", "service_id", "direction_id", "route_type"]]
    else:
        trip_route_merged = trip_route_merged[["trip_id", "route_id", "service_id", "route_type"]]

    if trips['trip_id'].dtype == object:

        # create a string version of trip_id
        stop_times['trip_id_str'] = stop_times['trip_id'].astype("str")

        # merge trip info onto stop_times
        stop_times_merged = stop_times[["trip_id_str", 'arrival_time', 'stop_id']].merge(trip_route_merged, right_on="trip_id", left_on="trip_id_str", how='left')

    else:

        stop_times_merged = stop_times[["trip_id", 'arrival_time', 'stop_id']].merge(trip_route_merged, right_on="trip_id", left_on="trip_id", how='left')

    # calculate a interpolated (float) time for future analysis


    if (sum(pd.isnull(stop_times_merged['arrival_time']))/len(stop_times_merged)) > 0.05:
        agency_trips = list(stop_times.drop_duplicates('trip_id')['trip_id'])
        allInterpolatedTimes = []
        for trip in agency_trips:
            allInterpolatedTimes += interpolator(trip)
        stop_times_merged['interpolated'] = allInterpolatedTimes
        #print("INTERP-O-LATED")
    else:
        stop_times_merged['interpolated'] = [minSinceMidnight(x) for x in stop_times_merged['arrival_time']]

    #stop_times_merged.head()
     
    # Dump all rail and ferry stops into their own CSVs. Remove rail and ferry stops from the <i>stop_times_merged</i> dataset.
    
    rail_times_merged = stop_times_merged.query("route_type in [0,1,2]")

    if len(rail_times_merged) > 0:
        rail_stops = pd.DataFrame(rail_times_merged['stop_id'].unique(), columns=['stop_id'])

        if stops['stop_id'].dtype != object:
            rail_stops = rail_stops.merge(stops[['stop_id', 'stop_name', 'stop_lon', 'stop_lat']], on="stop_id")
        else:
            rail_stops['stop_id_str'] = rail_stops['stop_id'].astype('str')
            rail_stops.drop("stop_id", axis=1, inplace=True)
            rail_stops = rail_stops.merge(stops[['stop_id', 'stop_name', 'stop_lon', 'stop_lat']], left_on="stop_id_str", right_on="stop_id")
            rail_stops.drop("stop_id_str", axis=1, inplace=True)
        rail_stops.to_csv("output/v3/rail/" + agency_name + ".csv")
    else:
        pass

    ferry_times_merged = stop_times_merged.query("route_type in [4]")

    if len(ferry_times_merged) > 0:
        ferry_stops = pd.DataFrame(ferry_times_merged['stop_id'].unique(), columns=['stop_id'])

        if stops['stop_id'].dtype != object:
            ferry_stops = ferry_stops.merge(stops[['stop_id', 'stop_name', 'stop_lon', 'stop_lat']], on="stop_id")
        else:
            ferry_stops['stop_id_str'] = ferry_stops['stop_id'].astype('str')
            ferry_stops.drop("stop_id", axis=1, inplace=True)
            ferry_stops = ferry_stops.merge(stops[['stop_id', 'stop_name', 'stop_lon', 'stop_lat']], left_on="stop_id_str", right_on="stop_id")
            ferry_stops.drop("stop_id_str", axis=1, inplace=True)
        ferry_stops.to_csv("output/v3/ferry/" + agency_name + ".csv")
    else:
        pass

    stop_times_merged = stop_times_merged.query("route_type == 3")
    
    # WEEKDAY

    if weekday in exceptions:
        weekday_stops = [weekday_service.remove(x) for x in exceptions[weekday][2] if x in weekday_service] + exceptions[weekday][1]
    else:
        pass

    weekday_stops = stop_times_merged[stop_times_merged['service_id'].isin(weekday_service)]
    weekday_stops.drop_duplicates(subset=["arrival_time", "stop_id"], inplace=True)
    #print(weekday_stops.shape)

    # SATURDAY

    if saturday in exceptions:
        saturday_service = [saturday_service.remove(x) for x in exceptions[saturday][2]] + exceptions[saturday][1]
    else:
        pass

    saturday_stops = stop_times_merged[stop_times_merged['service_id'].isin(saturday_service)]
    saturday_stops.drop_duplicates(subset=["arrival_time", "stop_id"], inplace=True)
    #print(saturday_stops.shape)
    
    # SUNDAY

    if sunday in exceptions:
        sunday_service = [sunday_service.remove(x) for x in exceptions[sunday][2]] + exceptions[sunday][1]
    else:
        pass
    
    sunday_stops = stop_times_merged[stop_times_merged['service_id'].isin(sunday_service)]
    sunday_stops.drop_duplicates(subset=["arrival_time", "stop_id"], inplace=True)    
    
    
    
    # OUTPUT AND ANALYSIS
    output_path = "output/v3/bus/" + agency_name + ".csv"

    if len(stop_times_merged) > 0:
        headways = pd.DataFrame(data=stops['stop_id'])
        #headways["stop_id"] = headways["stop_id"].astype(str)

        minutes = 60 * 4
        

        queries = [am_peak_query, pm_peak_query, weekday_range_query, saturday_range_query, sunday_range_query]
        periods = ['am_pk', 'pm_pk', 'wkdy', 'sat', 'sun']
        minute_ranges = [(600-360), (1140-900), (1320-360), (1320-480), (1320-480)]
        dfs = [weekday_stops, weekday_stops, weekday_stops, saturday_stops, sunday_stops]


        for i in range(5):

            # conditional parameters
            df = dfs[i]
            minutes = minute_ranges[i]
            query = queries[i]

            if "direction_id" in df.columns:
                for direction in [0,1]:
                    results = pd.DataFrame(minutes / df[df['direction_id']==direction].query(query)['stop_id'].value_counts())
                    results.reset_index(inplace=True)
                    results.rename(columns={"stop_id":periods[i]}, inplace=True)
                    results.rename(columns={"index":"stop_id"}, inplace=True)

                    results["stop_id"] = results["stop_id"].astype(str)
                    headways = headways.merge(results, on="stop_id", suffixes=["_dir0", "_dir1"], how='left')
                
            else:
                results = pd.DataFrame(minutes / df.query(query)['stop_id'].value_counts())
                results.reset_index(inplace=True)
                results.rename(columns={"stop_id":periods[i]}, inplace=True)
                results.rename(columns={"index":"stop_id"}, inplace=True)

                results["stop_id"] = results["stop_id"].astype(str)
                headways = headways.merge(results, on="stop_id", how='left')
        
        if "am_pk_dir0" in headways.columns:
            headways["am_pk"] = headways[["am_pk_dir0", "am_pk_dir1"]].min(axis=1)
            headways["pm_pk"] = headways[["pm_pk_dir0", "pm_pk_dir1"]].min(axis=1)

            headways["wkdy"] = headways[["wkdy_dir0", "wkdy_dir1"]].min(axis=1)
            headways["sat"] = headways[["sat_dir0", "sat_dir1"]].min(axis=1)
            headways["sun"] = headways[["sun_dir0", "sun_dir1"]].min(axis=1)
            headways = headways[['stop_id', "am_pk", "pm_pk", "wkdy", "sat", "sun"]] 
        else:
            pass
        #stops["stop_id"] = stops["stop_id"].astype(str)
        output = stops[["stop_id", "stop_name", "stop_lon", "stop_lat"]].merge(headways, on="stop_id", how="outer")
        output.fillna(9999, inplace=True)
        
        output.to_csv(output_path)
    else:
        print("No bus data.")
    
    hqt_filter = "(am_pk <= 15) & (pm_pk <= 15)"
    hqt_filter += " & (wkdy <= 20) & "
    hqt_filter += "(sat <= 30) & (sun <= 30)"
    
    
    hqt = output.query(hqt_filter)

    if len(hqt) > 0:
        hqt.to_csv("output/v3/bus-hqt/" + agency_name + " (HQT).csv")
    else:
        pass
# output.head()

errors_path = "output/v3/errors-{}-{}-{}.csv".format(weekday, saturday, sunday)
errors.to_csv(errors_path)
        

AC Transit 1 of 105
Airport Valet Express 2 of 105
Altamont Corridor Express 3 of 105
No bus data.
Amador Transit 4 of 105
Anaheim Resort Transportation 5 of 105
Eureka Transit Service 6 of 105
Avalon Transit 7 of 105
Bay Area Rapid Transit 8 of 105
No bus data.
San Francisco Bay Ferry 9 of 105
No bus data.
Calaveras Transit 10 of 105
Caltrain 11 of 105
Capitol Corridor Joint Powers Authority 12 of 105
eTrans 13 of 105
Glendale Beeline 14 of 105
City of Lodi Transit-GrapeLine 15 of 105
City of San Luis Obispo Transit 16 of 105
Big Blue Bus 17 of 105
TORRANCE TRANSIT SYSTEM 18 of 105
Commute.org Shuttle 19 of 105
Corona Cruiser 20 of 105
County Connection 21 of 105
Culver CityBus 22 of 105
Duarte Transit 23 of 105
Eastern Sierra Transit Authority 24 of 105
El Dorado Transit 25 of 105
El Monte Transit 26 of 105
Emery Go-Round 27 of 105
Fairfield and Suisun Transit 28 of 105
Foothill Transit 29 of 105
Fresno Public Transportation (FAX) 30 of 105
Fresno County Rural Transit Agency 31 of 10

KeyError: '20190201'

In [6]:
pd.read_csv(output_path)

,Unnamed: 0,stop_id,stop_name,stop_lon,stop_lat,am_pk,pm_pk,wkdy,sat,sun
0,0,0100060,8th St:Portola Av,-122.272992,37.768814,30.000000,15.000000,22.325581,30.000000,30.000000
1,1,0100070,8th St:Portola Av,-122.272798,37.768993,13.333333,30.000000,22.857143,30.000000,30.000000
2,2,0100110,1105 Atlantic Av,-122.264015,37.780272,20.000000,21.818182,24.615385,30.000000,30.000000
3,3,0100170,Atlantic Av:Challenger Dr,-122.270060,37.781066,20.000000,24.000000,24.000000,30.000000,30.000000
4,4,0100220,Atlantic Av:Triumph Dr,-122.266125,37.780799,20.000000,21.818182,24.615385,30.000000,30.000000
5,5,0100230,Atlantic Av:Triumph Dr,-122.265540,37.780605,20.000000,24.000000,24.000000,30.000000,30.000000
6,6,0100270,Aughinbaugh Way:Kofman Pkwy,-122.250125,37.741713,34.285714,30.000000,30.967742,31.111111,31.111111
7,7,0100280,Aughinbaugh Way:Mecartney Rd,-122.252140,37.738745,16.000000,24.000000,23.414634,30.000000,30.000000
8,8,0100290,Aughinbaugh Way:Mecartney Rd,-122.251960,37.738463,34.285714,30.000000,30.967742,31.111111,31.111111
9,9,0100300,Aughinbaugh Way:Robert Davey Jr Dr,-122.250108,37.742485,16.000000,24.000000,23.414634,30.000000,30.000000
